# QuakeML to json


<div class="alert alert-warning">

**Warning**: Much of this functionality will eventually be moved to obspy, see [this PR](https://github.com/obspy/obspy/pull/2210).

</div>

The following demonstrates obsplus' ability to serialize obspy catalog objects into json. All such conversions should be lossless.

## Get a catalog

In [ ]:
import obspy
import obsplus

crandall = obsplus.load_dataset('crandall')

cat = crandall.event_client.get_events()

print(cat)

## Convert to json

In [ ]:
import obsplus

# convert to json str
json_str = obsplus.cat_to_json(cat)

# print sample
print(json_str[0:400])

In [ ]:
# convert back, both catalogs should be equal
cat2 = obsplus.json_to_cat(json_str)
assert cat1 == cat2